In [20]:
# Import Libraries
import pandas as pd
import requests
import pymongo
from bs4 import BeautifulSoup as bs
import requests
#from api_keys import weather_api_key
import prettypandas
import json


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [21]:
from datetime import datetime

In [22]:
# import and set up CSV
schedule_csv = "Resources/NFLSchedule.csv"

full_schedule = pd.read_csv(schedule_csv)
full_schedule.head()

,Unnamed: 0,Year,Week,Day,Date,Time,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam
0,0,1967,1,Sun,September 17,NaN,Pittsburgh Steelers,Chicago Bears,41,13,393,2,95,6,Pittsburgh Steelers,Chicago Bears
1,1,1967,1,Sun,September 17,NaN,Baltimore Colts,Atlanta Falcons,38,31,508,2,303,5,Baltimore Colts,Atlanta Falcons
2,2,1967,1,Sun,September 17,NaN,Green Bay Packers,Detroit Lions,17,17,306,5,233,1,Green Bay Packers,Detroit Lions
3,3,1967,1,Sun,September 17,NaN,Philadelphia Eagles,Washington Redskins,35,24,426,2,388,1,Philadelphia Eagles,Washington Redskins
4,4,1967,1,Sun,September 17,NaN,San Francisco 49ers,Minnesota Vikings,27,21,360,1,335,3,Minnesota Vikings,San Francisco 49ers


In [23]:
# remove columns
removed_col_df = full_schedule[['Year','Week','Date','Time','Winner/tie','HomeTeam']]
removed_col_df.head()

,Year,Week,Date,Time,Winner/tie,HomeTeam
0,1967,1,September 17,NaN,Pittsburgh Steelers,Pittsburgh Steelers
1,1967,1,September 17,NaN,Baltimore Colts,Baltimore Colts
2,1967,1,September 17,NaN,Green Bay Packers,Green Bay Packers
3,1967,1,September 17,NaN,Philadelphia Eagles,Philadelphia Eagles
4,1967,1,September 17,NaN,San Francisco 49ers,Minnesota Vikings


In [24]:
full_date_df = removed_col_df
full_date_df["Full Date"] = full_schedule['Year'].map(str) + '-' + full_schedule["Date"].map(str)
full_date_df.head()


<ipython-input-24-71af9b739d5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_date_df["Full Date"] = full_schedule['Year'].map(str) + '-' + full_schedule["Date"].map(str)


,Year,Week,Date,Time,Winner/tie,HomeTeam,Full Date
0,1967,1,September 17,NaN,Pittsburgh Steelers,Pittsburgh Steelers,1967-September 17
1,1967,1,September 17,NaN,Baltimore Colts,Baltimore Colts,1967-September 17
2,1967,1,September 17,NaN,Green Bay Packers,Green Bay Packers,1967-September 17
3,1967,1,September 17,NaN,Philadelphia Eagles,Philadelphia Eagles,1967-September 17
4,1967,1,September 17,NaN,San Francisco 49ers,Minnesota Vikings,1967-September 17


In [25]:
full_date_df["Full Date"] = pd.to_datetime(full_date_df["Full Date"], format="%Y-%B %d")

<ipython-input-25-b138d2b2d18b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_date_df["Full Date"] = pd.to_datetime(full_date_df["Full Date"], format="%Y-%B %d")


In [26]:
full_date_df.head()

,Year,Week,Date,Time,Winner/tie,HomeTeam,Full Date
0,1967,1,September 17,NaN,Pittsburgh Steelers,Pittsburgh Steelers,1967-09-17
1,1967,1,September 17,NaN,Baltimore Colts,Baltimore Colts,1967-09-17
2,1967,1,September 17,NaN,Green Bay Packers,Green Bay Packers,1967-09-17
3,1967,1,September 17,NaN,Philadelphia Eagles,Philadelphia Eagles,1967-09-17
4,1967,1,September 17,NaN,San Francisco 49ers,Minnesota Vikings,1967-09-17


In [27]:
# filter for year 2007< since API historical can only go back that far
year_filter_df = full_date_df[full_date_df['Year']> 2007]
year_filter_df.head()

,Year,Week,Date,Time,Winner/tie,HomeTeam,Full Date
9136,2008,1,September 4,7:08PM,New York Giants,New York Giants,2008-09-04
9137,2008,1,September 7,1:02PM,Philadelphia Eagles,Philadelphia Eagles,2008-09-07
9138,2008,1,September 7,1:02PM,New York Jets,Miami Dolphins,2008-09-07
9139,2008,1,September 7,1:02PM,New England Patriots,New England Patriots,2008-09-07
9140,2008,1,September 7,1:02PM,Pittsburgh Steelers,Pittsburgh Steelers,2008-09-07


In [28]:
city_team = year_filter_df["HomeTeam"].astype(str)
city_team.head()

9136         New York Giants
9137     Philadelphia Eagles
9138          Miami Dolphins
9139    New England Patriots
9140     Pittsburgh Steelers
Name: HomeTeam, dtype: object

In [29]:
city_split = year_filter_df
city_split[["City","Home Team"]] = year_filter_df["HomeTeam"].str.rsplit(n=1, expand=True)
city_split[["Winning City","Winner"]] = year_filter_df["Winner/tie"].str.rsplit(n=1, expand=True)
city_split.head()

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,Week,Date,Time,Winner/tie,HomeTeam,Full Date,City,Home Team,Winning City,Winner
9136,2008,1,September 4,7:08PM,New York Giants,New York Giants,2008-09-04,New York,Giants,New York,Giants
9137,2008,1,September 7,1:02PM,Philadelphia Eagles,Philadelphia Eagles,2008-09-07,Philadelphia,Eagles,Philadelphia,Eagles
9138,2008,1,September 7,1:02PM,New York Jets,Miami Dolphins,2008-09-07,Miami,Dolphins,New York,Jets
9139,2008,1,September 7,1:02PM,New England Patriots,New England Patriots,2008-09-07,New England,Patriots,New England,Patriots
9140,2008,1,September 7,1:02PM,Pittsburgh Steelers,Pittsburgh Steelers,2008-09-07,Pittsburgh,Steelers,Pittsburgh,Steelers


In [12]:
#full_date_df["Full Date"] = pd.to_datetime(full_date_df["Full Date"], format="%Y-%B %d")
city_split["Time"] = pd.to_datetime(city_split["Time"],format="%S")

ValueError: unconverted data remains: :08PM

In [30]:
setup_df = city_split[["Full Date","Time",'Week',"City","Home Team","Winner"]]
setup_df.head()

,Full Date,Time,Week,City,Home Team,Winner
9136,2008-09-04,7:08PM,1,New York,Giants,Giants
9137,2008-09-07,1:02PM,1,Philadelphia,Eagles,Eagles
9138,2008-09-07,1:02PM,1,Miami,Dolphins,Jets
9139,2008-09-07,1:02PM,1,New England,Patriots,Patriots
9140,2008-09-07,1:02PM,1,Pittsburgh,Steelers,Steelers


In [31]:
# fix time to display in hhmm (1am is 100, 3pm is 1500)

In [32]:
final_table = setup_df.reset_index()
final_table

,index,Full Date,Time,Week,City,Home Team,Winner
0,9136,2008-09-04,7:08PM,1,New York,Giants,Giants
1,9137,2008-09-07,1:02PM,1,Philadelphia,Eagles,Eagles
2,9138,2008-09-07,1:02PM,1,Miami,Dolphins,Jets
3,9139,2008-09-07,1:02PM,1,New England,Patriots,Patriots
4,9140,2008-09-07,1:02PM,1,Pittsburgh,Steelers,Steelers
...,...,...,...,...,...,...,...
3199,12335,2019-01-12,3:05PM,Division,Kansas City,Chiefs,Chiefs
3200,12336,2019-01-12,6:40PM,Division,Green Bay,Packers,Packers
3201,12337,2019-01-19,3:05PM,ConfChamp,Kansas City,Chiefs,Chiefs
3202,12338,2019-01-19,6:40PM,ConfChamp,San Francisco,49ers,49ers


In [33]:
final_table = final_table[["Full Date","Week","Time","City","Home Team","Winner"]]
final_table
final_table.to_csv("NFL_Schedule.csv",encoding="utf-8",index=False)

In [34]:
playoffs = ["WildCard","Division","ConfChamp","SuperBowl"]
indoor_teams = ["Cowboys","Saints","Falcons","Vikings","Lions","Texans"]
indoor_city = ["Dallas","New Orleans", "Atlanta","Minnesota","Detroit","Houston"]

In [35]:
base_url = https://api.worldweatheronline.com/premium/v1/past-weather.ashx

SyntaxError: invalid syntax (<ipython-input-35-0a056756590b>, line 1)

In [ ]:
# http://www.worldweatheronline.com/feed/wwoConditionCodes.txt